Aug 6

In [26]:
path = 'data/TUM_RGBD/rgbd_dataset_freiburg1_desk/language_features_dim3/1305031452.791720_f.npy'

import numpy as np

np.load(path).shape

(156, 3)

### Jul 30

In [5]:
import numpy as np

load_path = '/mnt/c2d9b23a-b03e-4fdb-82ad-59f039ec9e3e/intern/King_Hang/OFFLINE-samslam_2/experiments/Replica/room0_0/params.npz'
data = np.load(load_path)

for key in data.keys():
    print(key)

means3D
rgb_colors
pointwise_le
unnorm_rotations
logit_opacities
log_scales
cam_unnorm_rots
cam_trans
timestep
intrinsics
w2c
org_width
org_height
gt_w2c_all_frames
keyframe_time_indices


In [16]:
import torch

query_tensor = torch.Tensor([[0.7379534, 0.6642408, -0.1192008]])

data['pointwise_le'].shape # (208205,9)
le_s = data['pointwise_le'][:,:3]

(208295, 3)

In [41]:
def get_cosine_similarity(query_tensor, le_s):

    # settings
    cos_threshold = 0.1

    # Convert numpy arrays to tensors if needed
    if isinstance(query_tensor, np.ndarray):
        query_tensor = torch.from_numpy(query_tensor)
    if isinstance(le_s, np.ndarray):
        le_s = torch.from_numpy(le_s)
    
    query_tensor = torch.tensor(query_tensor, dtype=torch.float32)
    le_s = torch.tensor(le_s, dtype=torch.float32)
    
    # check dimensions
    if query_tensor.dim() != 2 or le_s.dim() != 2:
        raise ValueError("Both query_tensor and le_s should be 2D tensors.")
    if query_tensor.size(1) != le_s.size(1):
        raise ValueError("The second dimension of query_tensor and le_s must be the same.")

    norm_query_tensor = query_tensor / query_tensor.norm(dim=1, keepdim=True)
    norm_le_s = le_s / le_s.norm(dim=1, keepdim=True)
    
    cosine_similarities = torch.mm(norm_query_tensor, norm_le_s.t())
    indices = (cosine_similarities.squeeze(0) > cos_threshold).nonzero(as_tuple=True)[0]
    
    return cosine_similarities, indices

le_s = data['pointwise_le'][:,:3]
cosine_similarities, indices = get_cosine_similarity(query_tensor, le_s)

/tmp/ipykernel_1182208/292529175.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_tensor = torch.tensor(query_tensor, dtype=torch.float32)
/tmp/ipykernel_1182208/292529175.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  le_s = torch.tensor(le_s, dtype=torch.float32)


In [46]:
le_s[indices]

array([[ 0.0874785 , -0.09367711, -1.1776744 ],
       [ 0.5383674 , -0.24182074, -0.86271596],
       [ 0.5749505 , -0.19785348, -0.8421425 ],
       ...,
       [ 0.58902586, -0.39352894, -0.7153767 ],
       [ 0.5780935 , -0.39636886, -0.7157943 ],
       [ 0.53266346, -0.44069973, -0.7087772 ]], dtype=float32)

## Previous

In [9]:
import glob
import os
from pathlib import Path
from typing import Dict, List, Optional, Union

import numpy as np
import torch
from natsort import natsorted
import os

In [10]:
input_folder = "./data/Replica/room0"
f_path = os.path.join(input_folder, 'language_features_dim3')

In [11]:
s_paths = natsorted(glob.glob(f"{input_folder}/language_features_dim3/*_s.npy"))
f_paths = natsorted(glob.glob(f"{input_folder}/language_features_dim3/*_f.npy"))

In [12]:
f_path = f_paths[0]
f1= torch.from_numpy(np.load(f_path))
f1.shape

torch.Size([150, 3])

In [13]:
s_path = s_paths[0]
s1= torch.from_numpy(np.load(s_path))
s1.shape

torch.Size([4, 680, 1200])

In [14]:
mask = seg != -1 # >> change to 0 later

mask.shape

torch.Size([4, 816000])

In [15]:
seg_map = s1
feature_map = f1

height, width = 680, 1200
le_dim = 3 #>> CLIP = 512, scenewise = 3
n_seg_lvls = 3

y, x = torch.meshgrid(torch.arange(0, height), torch.arange(0, width))
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)

seg = seg_map[:, y, x].squeeze(-1).long()
mask = seg != -1 # >> change to 0 later

all_lvl_feats = torch.zeros((n_seg_lvls, le_dim, height, width)) 
all_lvl_masks = torch.zeros((n_seg_lvls, 1, height, width))

for i in range(1, 4):  # only use s, m, l
    print('i : ')
    print(i)
    point_feature1 = feature_map[seg[i:i+1]].squeeze(0)
    point_feature = point_feature1.reshape(height, width, -1).permute(2, 0, 1)
    all_lvl_feats[i-1] = point_feature

    # get mask
    lvl_mask = mask[i:i+1].reshape(1, height, width)
    all_lvl_masks[i-1] = lvl_mask


i : 
1
i : 
2
i : 
3


In [19]:
feature_map.shape
seg_map.shape

torch.Size([4, 680, 1200])

In [16]:
import numpy as np
path = '/mnt/c2d9b23a-b03e-4fdb-82ad-59f039ec9e3e/intern/King_Hang/OFFLINE-samslam/experiments/Replica/room0_0/params10.npz'
obj = np.load(path)

print(obj.files)


obj['pointwise_le'][:10]

['means3D', 'rgb_colors', 'pointwise_le', 'unnorm_rotations', 'logit_opacities', 'log_scales', 'cam_unnorm_rots', 'cam_trans']


array([[-0.7695265 , -0.15192293, -0.6202809 , -0.46570873, -0.07955976,
        -0.8813544 , -0.6167774 , -0.07923762, -0.7305264 ],
       [-0.7695265 , -0.15192293, -0.6202809 , -0.46570873, -0.07955976,
        -0.8813544 , -0.6659762 , -0.02166895, -0.69057965],
       [-0.7695265 , -0.15192293, -0.6202809 , -0.46570873, -0.07955976,
        -0.8813544 , -0.65042406, -0.08929549, -0.69569665],
       [-0.7695265 , -0.15192293, -0.6202809 , -0.46570873, -0.07955976,
        -0.8813544 , -0.6451719 , -0.11420973, -0.6805587 ],
       [-0.7695265 , -0.15192293, -0.6202809 , -0.46570873, -0.07955976,
        -0.8813544 , -0.6837491 , -0.16649857, -0.7151868 ],
       [-0.7695265 , -0.15192293, -0.6202809 , -0.46570873, -0.07955976,
        -0.8813544 , -0.6958984 , -0.15972506, -0.684766  ],
       [-0.7695265 , -0.15192293, -0.6202809 , -0.46570873, -0.07955976,
        -0.8813544 , -0.70884556, -0.10204678, -0.69775265],
       [-0.7695265 , -0.15192293, -0.6202809 , -0.46570873, -0

Jul 29

In [17]:
import torch

# Load the dump file
dump_file = "buggy.dump"
try:
    data_0 = torch.load(dump_file)
    print("Contents of the dump file:")
    # print(data)
except Exception as e:
    print(f"Error loading the dump file: {e}")

Contents of the dump file:


In [19]:
args_list = [
    "raster_settings.bg",
    "means3D",
    "colors_precomp",
    "language_feature_precomp",
    "opacities",
    "scales",
    "rotations",
    "raster_settings.scale_modifier",
    "cov3Ds_precomp",
    "raster_settings.viewmatrix",
    "raster_settings.projmatrix",
    "raster_settings.tanfovx",
    "raster_settings.tanfovy",
    "raster_settings.image_height",
    "raster_settings.image_width",
    "sh",
    "raster_settings.sh_degree",
    "raster_settings.campos",
    "raster_settings.prefiltered",
    "raster_settings.debug",
    "raster_settings.include_feature"
]

for d, arg in zip(data_0, args_list):
    if isinstance(d, torch.Tensor):
        print(arg)
        print(d.shape)
    else:
        print(arg)
        print(d)
    print('\n')

raster_settings.bg
torch.Size([3])


means3D
torch.Size([950337, 3])


colors_precomp
torch.Size([950337, 3])


language_feature_precomp
torch.Size([1])


opacities
torch.Size([950337, 1])


scales
torch.Size([950337, 3])


rotations
torch.Size([950337, 4])


raster_settings.scale_modifier
1.0


cov3Ds_precomp
torch.Size([0])


raster_settings.viewmatrix
torch.Size([1, 4, 4])


raster_settings.projmatrix
torch.Size([1, 4, 4])


raster_settings.tanfovx
1.0


raster_settings.tanfovy
0.5666666666666667


raster_settings.image_height
680


raster_settings.image_width
1200


sh
torch.Size([0])


raster_settings.sh_degree
0


raster_settings.campos
torch.Size([3])


raster_settings.prefiltered
False


raster_settings.debug
True


raster_settings.include_feature
True




how does a normal looking file look?

In [14]:
dump_file = "snapshot_fw.dump"
try:
    data = torch.load(dump_file)
    print("Contents of the dump file:")
    print(data)
except Exception as e:
    print(f"Error loading the dump file: {e}")

Contents of the dump file:
(tensor([0., 0., 0.]), tensor([[-1.2046, -0.6822,  1.2056],
        [-1.2041, -0.6830,  1.2071],
        [-1.2035, -0.6838,  1.2085],
        ...,
        [ 1.6546,  0.9402,  1.6616],
        [ 1.6574,  0.9402,  1.6616],
        [ 1.6605,  0.9403,  1.6619]]), tensor([[1.2056, 1.0000, 1.4535],
        [1.2071, 1.0000, 1.4572],
        [1.2085, 1.0000, 1.4605],
        ...,
        [1.6616, 1.0000, 2.7608],
        [1.6616, 1.0000, 2.7608],
        [1.6619, 1.0000, 2.7618]]), tensor([0.]), tensor([[0.5000],
        [0.5000],
        [0.5000],
        ...,
        [0.5000],
        [0.5000],
        [0.5000]]), tensor([[0.0020, 0.0020, 0.0020],
        [0.0020, 0.0020, 0.0020],
        [0.0020, 0.0020, 0.0020],
        ...,
        [0.0028, 0.0028, 0.0028],
        [0.0028, 0.0028, 0.0028],
        [0.0028, 0.0028, 0.0028]]), tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [1., 0., 0., 

In [15]:
args_list = [
    "raster_settings.bg",
    "means3D",
    "colors_precomp",
    "language_feature_precomp",
    "opacities",
    "scales",
    "rotations",
    "raster_settings.scale_modifier",
    "cov3Ds_precomp",
    "raster_settings.viewmatrix",
    "raster_settings.projmatrix",
    "raster_settings.tanfovx",
    "raster_settings.tanfovy",
    "raster_settings.image_height",
    "raster_settings.image_width",
    "sh",
    "raster_settings.sh_degree",
    "raster_settings.campos",
    "raster_settings.prefiltered",
    "raster_settings.debug",
    "raster_settings.include_feature"
]

for d, arg in zip(data, args_list):
    if isinstance(d, torch.Tensor):
        print(arg)
        print(d.shape)
    else:
        print(arg)
        print(d)
    print('\n')

raster_settings.bg
torch.Size([3])


means3D
torch.Size([815998, 3])


colors_precomp
torch.Size([815998, 3])


language_feature_precomp
torch.Size([1])


opacities
torch.Size([815998, 1])


scales
torch.Size([815998, 3])


rotations
torch.Size([815998, 4])


raster_settings.scale_modifier
1.0


cov3Ds_precomp
torch.Size([0])


raster_settings.viewmatrix
torch.Size([1, 4, 4])


raster_settings.projmatrix
torch.Size([1, 4, 4])


raster_settings.tanfovx
1.0


raster_settings.tanfovy
0.5666666666666667


raster_settings.image_height
680


raster_settings.image_width
1200


sh
torch.Size([0])


raster_settings.sh_degree
0


raster_settings.campos
torch.Size([3])


raster_settings.prefiltered
False


raster_settings.debug
True


raster_settings.include_feature
True


